In [1]:
import pickle
import pandas as pd

# Load saved pipeline components
with open('fertilizer_model_pipeline.pkl', 'rb') as f:
    saved_objects = pickle.load(f)

best_rf = saved_objects['model']
pt = saved_objects['scaler']
le_crop = saved_objects['le_crop']
le_soil = saved_objects['le_soil']
le_fert = saved_objects['le_fert']


In [2]:
def recommend_fertilizer(input_data):
    """
    input_data: dict with keys: Nitrogen, Phosphorous, Potassium, PH, Temperature,
                                Moisture, Rainfall, Carbon, Crop, Soil
    Returns fertilizer recommendation string.
    """

    # Create DataFrame from input dictionary
    df_input = pd.DataFrame([input_data])

    # Encode categorical variables
    df_input['Crop_le'] = le_crop.transform(df_input['Crop'])
    df_input['Soil_le'] = le_soil.transform(df_input['Soil'])

    # List of numeric columns
    numeric_cols = ['Nitrogen', 'Phosphorous', 'Potassium', 'PH', 'Temperature',
                    'Moisture', 'Rainfall', 'Carbon']

    # Scale numeric features
    df_input[numeric_cols] = pt.transform(df_input[numeric_cols])

    # Select features for prediction
    features = numeric_cols + ['Crop_le', 'Soil_le']
    X_input = df_input[features]

    # Predict encoded fertilizer label
    pred_encoded = best_rf.predict(X_input)[0]

    # Decode fertilizer label
    pred_fertilizer = le_fert.inverse_transform([pred_encoded])[0]

    return pred_fertilizer


In [ ]:
# Example input dictionary
sample_input = {
    'Nitrogen': 54.14082354, 
    'Phosphorous': 134.7425253,
    'Potassium': 159.5640307,
    'PH': 3.908612908,
    'Temperature': 28.9891225, # in celcius
    'Moisture': 0.460655459, # soil moisture, standard soil moisture unit
    'Rainfall': 32.87016218, #in mm
    'Carbon': 0.800157046, #in %-age
    'Crop': 'Mung Bean', 
    'Soil': 'Neutral Soil' #dropdown-> Acidic Soil , Peaty Soil , Neutral Soil  , Loamy Soil  ,Alkaline Soil 
}
#result:  Lime
recommendation = recommend_fertilizer(sample_input)
print("Recommended Fertilizer:", recommendation)

#crop dropdown -> rice, wheat, Mung Bean,  Tea , millet, maize ,Lentil, Jute,  Coffee, Cotton, Ground Nut, Peas ,Rubber ,Sugarcane ,Tobacco ,
# KidneyBeans, Moth Beans 
#Coconut , Black gram ,Adzuki, Beans ,Pigeon Peas, Chickpea, banana, grapes, apple, mango ,muskmelon, orange, papaya, pomegranate, watermelon 

Recommended Fertilizer: Lime


In [4]:

df=pd.read_csv('fertilizer_cleaned.csv')
df.head()


,Temperature,Moisture,Rainfall,PH,Nitrogen,Phosphorous,Potassium,Carbon,Soil,Crop,Fertilizer,Remark,Crop_le,Soil_le,Fertilizer_le
0,50.179845,0.725893,205.600816,6.227358,66.701872,76.963560,96.429065,0.496300,Loamy Soil,rice,Compost,Enhances organic matter and improves soil stru...,28,2,1
1,21.633318,0.721958,306.081601,7.173131,71.583316,163.057636,148.128347,1.234242,Loamy Soil,rice,Balanced NPK Fertilizer,"Provides a balanced mix of nitrogen, phosphoru...",28,2,0
2,23.060964,0.685751,259.336414,7.380793,75.709830,62.091508,80.308971,1.795650,Peaty Soil,rice,Water Retaining Fertilizer,Improves water retention in dry soils. Prefer ...,28,4,9
3,26.241975,0.755095,212.703513,6.883367,78.033687,151.012521,153.005712,1.517556,Loamy Soil,rice,Balanced NPK Fertilizer,"Provides a balanced mix of nitrogen, phosphoru...",28,2,0
4,21.490157,0.730672,268.786767,7.578760,71.765123,66.257371,97.000886,1.782985,Peaty Soil,rice,Organic Fertilizer,"Enhances fertility naturally, ideal for peaty ...",28,4,7
